In [ ]:
import torch
from torch.autograd import Variable
import torch.nn.functional as F
import torch.utils.data as Data

import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
import pandas as pd   
import re
import os   

In [ ]:
from google.colab import drive
#drive.flush_and_unmount()
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
cd "/content/drive/My Drive/Data Science Course/semester 2/NLP/homework/HW3 - semantic similarity"

/content/drive/My Drive/Data Science Course/semester 2/NLP/homework/HW3 - semantic similarity


In [ ]:
data = pd.read_csv("labeled_data.csv")

In [ ]:
data.head()

,Unnamed: 0,text_1,text_2,label,POS dist score,1-gram_overlap,chrf_score_norm,WMD,ROUGE-1 recall,ROUGE-1 precision,ROUGE-1 F,ROUGE-2 recall,ROUGE-2 precision,ROUGE-2 F,ROUGE-L recall,ROUGE-L precision,ROUGE-L F,BertScore,L2_score
0,0,take measures in order to,take additional measures to,4.000,5.000000,3.333333,2.533040,1.382194,3.000000,3.750000,3.333333,0.0,0.0,0.0,3.000000,3.750000,3.333333,4.5364,2.621138
1,1,was seeking to,will strive to,3.000,0.000000,1.666667,0.869565,3.197893,1.666667,1.666667,1.666667,0.0,0.0,0.0,1.666667,1.666667,1.666667,4.5531,2.147480
2,2,canada aims to,canada works to,2.333,5.000000,3.333333,2.658046,2.067205,3.333333,3.333333,3.333333,0.0,0.0,0.0,3.333333,3.333333,3.333333,4.7772,2.539115
3,3,refugees and,refugee population,2.667,2.305241,0.000000,2.475248,2.630209,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,4.3771,0.842952
4,4,measures and,"instruments , and",3.000,2.688024,2.500000,1.500000,2.365262,2.500000,1.666667,2.000000,0.0,0.0,0.0,2.500000,1.666667,2.000000,4.4446,1.865555


In [ ]:
data.drop(columns=["Unnamed: 0","text_1","text_2"],inplace=True)

In [ ]:
data.head()

,label,POS dist score,1-gram_overlap,chrf_score_norm,WMD,ROUGE-1 recall,ROUGE-1 precision,ROUGE-1 F,ROUGE-2 recall,ROUGE-2 precision,ROUGE-2 F,ROUGE-L recall,ROUGE-L precision,ROUGE-L F,BertScore,L2_score
0,4.000,5.000000,3.333333,2.533040,1.382194,3.000000,3.750000,3.333333,0.0,0.0,0.0,3.000000,3.750000,3.333333,4.5364,2.621138
1,3.000,0.000000,1.666667,0.869565,3.197893,1.666667,1.666667,1.666667,0.0,0.0,0.0,1.666667,1.666667,1.666667,4.5531,2.147480
2,2.333,5.000000,3.333333,2.658046,2.067205,3.333333,3.333333,3.333333,0.0,0.0,0.0,3.333333,3.333333,3.333333,4.7772,2.539115
3,2.667,2.305241,0.000000,2.475248,2.630209,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,4.3771,0.842952
4,3.000,2.688024,2.500000,1.500000,2.365262,2.500000,1.666667,2.000000,0.0,0.0,0.0,2.500000,1.666667,2.000000,4.4446,1.865555


In [ ]:
torch.manual_seed(1)    # reproducible
x = torch.tensor(np.array(data.iloc[:,1:]), dtype=torch.float32) #take only features
y = torch.tensor(np.array(data.iloc[:,0]), dtype=torch.float32) #take only lables 

In [ ]:
x.shape , y.shape

(torch.Size([998, 15]), torch.Size([998]))

In [ ]:
net = torch.nn.Sequential(
        torch.nn.Linear(x.shape[1], 50),
        torch.nn.LeakyReLU(),
        torch.nn.Linear(50, 100),
        torch.nn.LeakyReLU(),
        torch.nn.Linear(100, 1),
    )

In [ ]:
optimizer = torch.optim.Adam(net.parameters(), lr=0.01)
loss_func = torch.nn.MSELoss()  # this is for regression mean squared loss

In [ ]:
BATCH_SIZE = 4
EPOCH = 200


train_torch_dataset = Data.TensorDataset(x[:750, :], y[:750])
test_torch_dataset = Data.TensorDataset(x[750:, :], y[750:])

train_loader = Data.DataLoader(dataset=train_torch_dataset, batch_size=BATCH_SIZE,  shuffle=True, num_workers=2,)
test_loader = Data.DataLoader(dataset=test_torch_dataset, batch_size=BATCH_SIZE,  shuffle=True, num_workers=2,)


In [ ]:
# start training
for epoch in range(EPOCH):
  training_loss = 0
  for step, (batch_x, batch_y) in enumerate(train_loader): # for each training step
          
        
    b_x = Variable(batch_x)
    b_y = Variable(batch_y)

    prediction = net(b_x)     # input x and predict based on x
    loss = loss_func(prediction, b_y)     # must be (1. nn output, 2. target)

    optimizer.zero_grad()   # clear gradients for next train
    loss.backward()         # backpropagation, compute gradients
    optimizer.step()        # apply gradients

    training_loss += loss.item()
  print(f"Training loss: {training_loss/len(train_loader)}")


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:432: UserWarning: Using a target size (torch.Size([4])) that is different to the input size (torch.Size([4, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Training loss: 1.0031059851196218


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:432: UserWarning: Using a target size (torch.Size([2])) that is different to the input size (torch.Size([2, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Training loss: 0.9052119236835774
Training loss: 0.7653577500518333
Training loss: 0.720690657563032
Training loss: 0.7383819633222958
Training loss: 0.7095212718779023
Training loss: 0.7066368588464375
Training loss: 0.711865505381943
Training loss: 0.7247187855078819
Training loss: 0.6951543541347727
Training loss: 0.6902094496057388
Training loss: 0.6630028595236388
Training loss: 0.6666087029164934
Training loss: 0.668081790446601
Training loss: 0.6789241457863294
Training loss: 0.6547832225568275
Training loss: 0.643936472984546
Training loss: 0.6704676766146688
Training loss: 0.64502935936833
Training loss: 0.6825262764270635
Training loss: 0.6476007602474791
Training loss: 0.65065021876325
Training loss: 0.6444449744960095
Training loss: 0.6800768067663971
Training loss: 0.6535788015165227
Training loss: 0.6801383372079185
Training loss: 0.640599477379781
Training loss: 0.6561574806716848
Training loss: 0.6842093424217657
Training loss: 0.6547619339197557
Training loss: 0.645100

In [ ]:
# start testing

testing_loss = 0
for step, (batch_x, batch_y) in enumerate(test_loader): # for each training step
        
  b_x = Variable(batch_x)
  b_y = Variable(batch_y)

  prediction = net(b_x)     # input x and predict based on x
  loss = loss_func(prediction, b_y)     # must be (1. nn output, 2. target)

  testing_loss += loss.item()
print(f"testing loss: {testing_loss/len(test_loader)}")

testing loss: 0.6339470339157889


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:432: UserWarning: Using a target size (torch.Size([4])) that is different to the input size (torch.Size([4, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [ ]:

w = list(net.parameters())
w

[Parameter containing:
 tensor([[ 1.6080e-01, -4.7701e-01, -1.1910e+00,  3.7882e-01, -2.6354e-03,
          -7.6638e-02, -1.4267e-01,  6.8461e-01,  3.2823e-01,  2.2673e-01,
           1.0925e-02, -5.2119e-01, -2.9785e-01, -8.9852e-01, -1.1012e+00],
         [-1.1339e+00, -1.2170e-01, -7.5101e-01, -1.2713e-01,  2.4377e-01,
          -2.3888e-01, -1.7110e-01, -3.8567e-01, -7.3630e-01, -6.1386e-01,
           8.9916e-02, -6.5075e-02,  4.1878e-02, -8.5701e-01, -5.6861e-01],
         [-2.3527e-01,  2.9357e-02, -6.2764e-01,  3.6230e-01, -9.5669e-02,
          -3.1904e-01,  2.1574e-01, -4.8795e-02, -2.2181e-01, -3.7442e-01,
           2.6746e-01, -4.4817e-01,  3.9956e-02, -5.0411e-01, -5.1923e-01],
         [-3.7311e-01, -2.3439e-01, -1.6399e+00,  2.5981e-01,  7.4721e-02,
          -9.5946e-02, -2.9540e-02, -3.3854e-02,  2.6345e-01,  3.6847e-01,
           2.0863e-01,  4.8608e-02, -1.7520e-01, -9.7328e-01, -1.2328e+00],
         [-3.5793e-01, -1.7657e-01, -7.6195e-01, -3.6418e-01, -2.7561e-01

In [ ]:
net.modules

<bound method Module.modules of Sequential(
  (0): Linear(in_features=15, out_features=50, bias=True)
  (1): LeakyReLU(negative_slope=0.01)
  (2): Linear(in_features=50, out_features=100, bias=True)
  (3): LeakyReLU(negative_slope=0.01)
  (4): Linear(in_features=100, out_features=1, bias=True)
)>

In [ ]:
net[0]

Linear(in_features=15, out_features=50, bias=True)

In [ ]:
list(net[0].parameters())

[Parameter containing:
 tensor([[ 1.6080e-01, -4.7701e-01, -1.1910e+00,  3.7882e-01, -2.6354e-03,
          -7.6638e-02, -1.4267e-01,  6.8461e-01,  3.2823e-01,  2.2673e-01,
           1.0925e-02, -5.2119e-01, -2.9785e-01, -8.9852e-01, -1.1012e+00],
         [-1.1339e+00, -1.2170e-01, -7.5101e-01, -1.2713e-01,  2.4377e-01,
          -2.3888e-01, -1.7110e-01, -3.8567e-01, -7.3630e-01, -6.1386e-01,
           8.9916e-02, -6.5075e-02,  4.1878e-02, -8.5701e-01, -5.6861e-01],
         [-2.3527e-01,  2.9357e-02, -6.2764e-01,  3.6230e-01, -9.5669e-02,
          -3.1904e-01,  2.1574e-01, -4.8795e-02, -2.2181e-01, -3.7442e-01,
           2.6746e-01, -4.4817e-01,  3.9956e-02, -5.0411e-01, -5.1923e-01],
         [-3.7311e-01, -2.3439e-01, -1.6399e+00,  2.5981e-01,  7.4721e-02,
          -9.5946e-02, -2.9540e-02, -3.3854e-02,  2.6345e-01,  3.6847e-01,
           2.0863e-01,  4.8608e-02, -1.7520e-01, -9.7328e-01, -1.2328e+00],
         [-3.5793e-01, -1.7657e-01, -7.6195e-01, -3.6418e-01, -2.7561e-01

In [ ]:
net[0].weight.data.shape

torch.Size([50, 15])

In [ ]:
net[0].weight.data[0,:]

tensor([ 0.1608, -0.4770, -1.1910,  0.3788, -0.0026, -0.0766, -0.1427,  0.6846,
         0.3282,  0.2267,  0.0109, -0.5212, -0.2979, -0.8985, -1.1012])

In [ ]:
data.columns[1:].tolist()

['POS dist score',
 '1-gram_overlap',
 'chrf_score_norm',
 'WMD',
 'ROUGE-1 recall',
 'ROUGE-1 precision',
 'ROUGE-1 F',
 'ROUGE-2 recall',
 'ROUGE-2 precision',
 'ROUGE-2 F',
 'ROUGE-L recall',
 'ROUGE-L precision',
 'ROUGE-L F',
 'BertScore',
 'L2_score']

In [ ]:
features_weights = pd.DataFrame(data.columns[1:].tolist(), columns = ['features']) 

In [ ]:
features_weights["weights"] = net[0].weight.data[0,:]

In [ ]:
features_weights.sort_values(by=['weights'])

,features,weights
2,chrf_score_norm,-1.190981
14,L2_score,-1.101248
13,BertScore,-0.898518
11,ROUGE-L precision,-0.521188
1,1-gram_overlap,-0.477014
12,ROUGE-L F,-0.297854
6,ROUGE-1 F,-0.142669
5,ROUGE-1 precision,-0.076638
4,ROUGE-1 recall,-0.002635
10,ROUGE-L recall,0.010925
